### Ok, the first requirement is 

1. La universidad opera de forma presencial y con planes de estudios semestrales. Se debe registrar la información de cada plan de estudios como: nombre, clave, duración en semestres, número de créditos totales, estatus (vigente o no).

This seems to just be a table. The format is

CREATE TABLE study_plan(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    code VARCHAR(50),
    duration INTEGER,
    credits INTEGER,
    status BOOL,
);

I don't think code and ID are the same, as the code probably has implicit information about what subject is related to, the undergraduate that is imparted in, etc. As it probably has both letters and numbers I think it is better to just leave it as VARCHAR. 

I don't know if you declare a boolean variable in postgreSQL with BOOL, but I will know once I run the code. 

As this table doesn't has any foreign key is a strong entity; one of the first ones I will create in the database. 

### The second requirement is 

2. Se debe almacenar la información de todas las materias comprendidas en cada plan de estudios: clave, nombre, número de horas, número de horas teóricas, numéro de horas de práctica, número de créditos, área de especialización.

CREATE TABLE subjects(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    hours INTEGER,
    theorical_hours INTEGER,
    practical_hours INTEGER,
    credits INTEGER,
    area 
);

In area I will most probably need a foreign key. It seems that the relationship is one to one. I think that I should put the rule here that no attribute of this table can be null, but also that the arithmetic sum of the 'theorical_hours' and 'practical_hours' must be equal or less than 'hours'. 

### The third requirement is 

3. Se debe almacenar la información personal del alumno como: nombre completo, nacionalidad, curp, fecha de nacimiento, correo electrónico personal, teléfono, dirección, estado civil, tipo de sangre, numero de seguridad social. No es posible que dos estudiantes compartan correo electrónico.

Another table

CREATE TABLE students(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100),
    nacionality VARCHAR(10),
    curp VARCHAR(30), 
    birth_date DATE,
    email TEXT UNIQUE,
    telephone VARCHAR(15),
    city VARCHAR(100), 
    state VARCHAR(100), 
    street VARCHAR(100),
    external_number INTEGER,
    zip_code INTEGER,
);

Creating tables for the last names seems excessive, but useful at the same time... not really, I don't think having separate tables for the last names will help more than just doing a normal SQL search. My doubt is how can I ensure that the students will fill out the information in the 'Name PaternalName MaternalName' standard? 

In nacionality I have doubt too. I should probably have a list of codes that map all the nacionalities in the world, and nationality as an attribute to just be a number. 

The same goes for the CURP... I could use regex to ensure at least the curp that the student wrote makes sense, as I have already done it for the subject of formal languages. 

I will stop worrying for that now, as it seems that I can get into the same trouble with all the students' attributes. 

I know I need to pay special attention to the adress in order to comply with 3rd Normal Form. It seems that the way to do so is by breaking adress into composite values such as street, external number and zip code. 

It seems that for email I need to use TEXT in addition to a CHECK constraint. Indeed I can use regex for the email or other camps. 

As I have already wrote quite a bit of text I will start with the code. First importing psycopg2 for postgreSQL. 















In [1]:
import psycopg
from dotenv import load_dotenv
import os

load_dotenv()

def conn_to_postgres(query, params = None, fetch = False):
    conn = None

    try:
        conn = psycopg.connect(
            host="localhost",
            dbname="unam_database",
            user="schronding",
            password=os.getenv('PASSWORD'),
            port="5432", 
        ) 

        conn.autocommit = True
        cur = conn.cursor()

        cur.execute(query, params)

        if fetch:
            result = cur.fetchone()[0]
            cur.close()
            conn.close()
            return result
            
        results = cur.fetchall()

        for row in results:
            print(row)

        cur.close()
        return conn
    
    except(Exception, psycopg.DatabaseError) as e:
        print(f"Error: {e}")
        if conn:
            cur.close()
        return None



Before executing the function (which I think it is wrong as I probably confuse what the difference between the connection and cursor is), I need to create the database... but I don't remember the syntax 

**postgres=# CREATE DABATASE unam_database;**

*ERROR:  syntax error at or near "DABATASE"*

*LINE 1: CREATE DABATASE unam_database;*

I wrote DATABASE wrong... Now it works. 

It seems that while the syntax is correct I am not able to set schronding as the owner of 'unam_database'. I think it probably has to do with me not creating this role for this specific database. 

postgres=# CREATE DABATASE unam_database;

ERROR:  syntax error at or near "DABATASE"

LINE 1: CREATE DABATASE unam_database;

To check the roles I can use \du and \du+. 

I am having trouble now even entering to the postgres terminal 

**sudo psql -U postgres**

psql: error: connection to server on socket "/var/run/postgresql/.s.PGSQL.5432" failed: FATAL:  Peer authentication failed for user "postgres"

It seems that I can reconfigure the 'pg_hba.conf' file to trust my local computer... but I don't know where is located. It doesn't work with 'whereis' on the terminal. 

I will just use sudo -u postgres psql. 

It seems I am still denied permision but the terminal appeared again

**sudo -u postgres psql**

could not change directory to "/home/schronding/repos/UNAM_database": 

Permission denied

psql (15.14 (Debian 15.14-0+deb12u1))

It wasn't that my role didn't exist, as it already does 

**postgres=# CREATE ROLE schronding;**

ERROR:  role "schronding" already exists

With the command 

**ALTER ROLE schronding WITH LOGIN PASSWORD MarioYChuyBFF**

the postgres terminal changed from 'postgres=#' to 'postgres-#'. I don't know what it means the change from = to -. It seems that the difference is that when it goes to - it means that you haven't finished executing the previous command. I think this could be useful to create tables with multiple attributes. 

I think I should probably use the library of running terminal scripts in python, as I think that would cut the need to copy all my commands and their output. The library's name is 'subprocess'. It might not work, as there are two levels of 'depth' with the postgres terminal. 

In [2]:
import subprocess

study_plan_table = """
DROP TABLE IF EXISTS study_plans CASCADE; 

CREATE TABLE study_plans(
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) UNIQUE,
    code VARCHAR(50),
    duration INTEGER,
    credits INTEGER,
    status BOOLEAN NOT NULL
);"""

conn_to_postgres(study_plan_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


I had some errors

Error: invalid connection option "name"

Multiple connection attempts failed. All failures were:
- host: 'localhost', port: '5432', hostaddr: '::1': invalid connection option "name"

- host: 'localhost', port: '5432', hostaddr: '127.0.0.1': invalid connection option "name"

The thing with psycopg is that it seems that the order of the attributes matter, even though I am explicitly telling the parameters by name. While psycopg seems to be a better version than psycopg2 I might be getting all of these problems just by the difference in versions, as the conn that I remembered was with psycopg2. 

Also prior to that I forgot that in the except clause we need 'Exception', not 'Error'. Also that the port must be a string, not a number. 

It is interesting, because when I put name it throwed an error, when I changed the attribute to database it did too. It seems that the attribute itself is the problem. 

I don't know what the problem is... 

It was just that the correct name attribute was 'dbname'... 

At least now I get a different error, which I understand

Error: connection failed: connection to server at "127.0.0.1", port 5432 failed: fe_sendauth: no password supplied
Multiple connection attempts failed. All failures were:
- host: 'localhost', port: '5432', hostaddr: '::1': connection failed: connection to server at "::1", port 5432 failed: fe_sendauth: no password supplied
- host: 'localhost', port: '5432', hostaddr: '127.0.0.1': connection failed: connection to server at "127.0.0.1", port 5432 failed: fe_sendauth: no password supplied

What I don't understand is why the password isn't correctly retrieved. Could it be that by being in a Jupyter notebook, the getenv method doesn't recognize that there is a hidden file .env in the local environment? 

Indeed when I try to get the password it doesn't retrieve it

In [3]:
#print(os.getenv("PASSWORD"))

I now see what the problem was... I didn't include the 'load_dotenv()' function. 

Now it finally works 

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f52178c0>

In [4]:
print(conn_to_postgres("SELECT ALL"))

()
<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e2ca3250>


What I wanted to see was if there was a good formated answer for when I made a SQL request. I will research it later, I think I can continue with the second table. Thinking it more slowly I do think that while on the ENES there might be different versions of the same subject, for example 'Probabilidad y Estadistica' for technology and other for aeroespacial engineering, I think it makes sense that one subject can belong to many study plans, creating a connection one to many. 

Part of me thinks that many to many is correct, but is redundant. Can one study plan be involved in a lot of subjects? Yes, but is it relevant? Do I need to write the intermediate table? For now I will do it. 

As the intermediate or pivot table has foreign keys is weak, but subjects is weak too. Is there any difference by which I create first? 

In [5]:
plan_subjects_pivot_table = """
DROP TABLE IF EXISTS plan_subjects CASCADE;

CREATE TABLE plan_subjects(
    plan_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (plan_id) REFERENCES study_plans(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
);
"""

conn_to_postgres(plan_subjects_pivot_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


As I got the error

Error: syntax error at or near "FOREIGN"
LINE 5:     plan_id FOREIGN KEY id REFERENCES plan(id),
                    ^

I will try to first create the subjects table without the foreign key, and then execute the pivot table. 

In [6]:
subjects_table = """
DROP TABLE IF EXISTS subjects CASCADE;

CREATE TABLE subjects(
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100) UNIQUE,
    theorical_hours INTEGER,
    practical_hours INTEGER,
    hours INTEGER,
    credits INTEGER,
    area VARCHAR(100),
    career INTEGER,
    FOREIGN KEY (career) REFERENCES study_plans(id),
    CONSTRAINT hours CHECK (theorical_hours + practical_hours <= hours)
);"""

conn_to_postgres(subjects_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


It seems I got the error 

Error: syntax error at or near ")"
LINE 11: );
         ^

Because I left the last attribute with a comma at the end. When I removed it I got a sucessful message again. 

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f52db9b0>

The error remains in the pivot table though. I will continue with the students table while I think how to solve that error

In [7]:
students_table ="""
DROP TABLE IF EXISTS students CASCADE;

CREATE TABLE students(
    id SERIAL PRIMARY KEY,
    first_names VARCHAR(100),
    paternal_surname VARCHAR(100),
    maternal_surname VARCHAR(100), 
    nationality VARCHAR(2),
    curp VARCHAR(30), 
    birth_date DATE,
    email TEXT UNIQUE,
    telephone VARCHAR(15),
    city VARCHAR(100), 
    state VARCHAR(100), 
    street VARCHAR(100),
    external_number INTEGER,
    zip_code INTEGER,
    marital_status MARITALSTATUS,
    blood_type BLOODTYPE,
    NSS INTEGER,
    tutor_id INTEGER,
    FOREIGN KEY (tutor_id) REFERENCES professors(id),
    FOREIGN KEY (nationality) REFERENCES nationalities(code)
);"""

conn_to_postgres(students_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


It seems it worked! 

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f13f41456a0>

As I don't like that nacionality is VARCHAR I will use the lookup table of formal languages so I can change the type to a number. 

### Nacionalities

```Java
        paises.add("MX : MÉXICO");
        paises.add("DE : ALEMANIA");
        paises.add("AD : ANDORRA");
        paises.add("AR : ARGENTINA");
        paises.add("AU : AUSTRALIA");
        paises.add("AT : AUSTRIA");
        paises.add("BE : BÉLGICA");
        paises.add("BO : BOLIVIA");
        paises.add("BR : BRASIL");
        paises.add("CA : CANADÁ");
        paises.add("CL : CHILE");
        paises.add("CN : CHINA");
        paises.add("CO : COLOMBIA");
        paises.add("KR : COREA DEL SUR");
        paises.add("CR : COSTA RICA");
        paises.add("CU : CUBA");
        paises.add("DK : DINAMARCA");
        paises.add("EC : ECUADOR");
        paises.add("EG : EGIPTO");
        paises.add("SV : SALVADOR, EL");
        paises.add("ES : ESPAÑA");
        paises.add("US : ESTADOS UNIDOS DE AMÉRICA");
        paises.add("PH : FILIPINAS");
        paises.add("FI : FINLANDIA");
        paises.add("FR : FRANCIA");
        paises.add("GR : GRECIA");
        paises.add("GT : GUATEMALA");
        paises.add("HN : HONDURAS");
        paises.add("IN : INDIA");
        paises.add("ID : INDONESIA");
        paises.add("IE : IRLANDA");
        paises.add("IL : ISRAEL");
        paises.add("IT : ITALIA");
        paises.add("JM : JAMAICA");
        paises.add("JP : JAPON");
        paises.add("MA : MARRUECOS");
        paises.add("NI : NICARAGUA");
        paises.add("NO : NORUEGA");
        paises.add("NZ : NUEVA ZELANDA");
        paises.add("PA : PANAMA");
        paises.add("PY : PARAGUAY");
        paises.add("PE : PERÚ");
        paises.add("PL : POLONIA");
        paises.add("PT : PORTUGAL");
        paises.add("PR : PUERTO RICO");
        paises.add("GB : REINO UNIDO");
        paises.add("CZ : CHECA, REPÚBLICA");
        paises.add("DO : DOMICANA, REPÚBLICA");
        paises.add("RU : RUSIA");
        paises.add("ZA : SUDÁFRICA");
        paises.add("SE : SUECIA");
        paises.add("CH : SUIZA");
        paises.add("TH : TAILANDIA");
        paises.add("TR : TURQUÍA");
        paises.add("UA : UCRANIA");
        paises.add("UY : URUGUAY");
        paises.add("VE : VENEZUELA");
```

### Regex 

While it is in Java I will store here the regex in case they come in handy later down the line 

```Java
static {
        REGEX_MAP.put("Nombre(s)", "^[A-ZÁÉÍÓÚÑ ]{1,20}$");
        REGEX_MAP.put("Apellido Paterno", "^[A-ZÁÉÍÓÚÑ ]{1,15}$");
        REGEX_MAP.put("Apellido Materno", "^[A-ZÁÉÍÓÚÑ ]{1,15}$");
        REGEX_MAP.put("Fecha de nacimiento", "^(?:0?[1-9]|[12]\\d|3[01])([-/])(?:0?[1-9]|1[0-2])\\1(?:(?:\\d{2})?\\d{2})$");
        REGEX_MAP.put("RFC", "^[A-Z]{4}\\d{6}[A-Z0-9]{3}$");
        REGEX_MAP.put("CURP", "^[A-Z]{4}\\d{6}[HM][A-Z]{2}[A-Z]{3}[A-Z0-9]{2}$");
        REGEX_MAP.put("Calle", "^[A-Z0-9 ]+$");
        REGEX_MAP.put("Número Ext.", "^\\d+$");
        REGEX_MAP.put("Colonia", "^[A-Z0-9 ]+$");
        REGEX_MAP.put("Código Postal", "^\\d{5}$");
        REGEX_MAP.put("Teléfono1", "^\\d{3}$");
        REGEX_MAP.put("Teléfono2", "^\\d{3}$");
        REGEX_MAP.put("Teléfono3", "^\\d{4}$");
        REGEX_MAP.put("Correo electrónico", "^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9]+(?:[.-][a-zA-Z0-9]+)*\\.[a-zA-Z]{2,}$");
        REGEX_MAP.put("Contraseña", "^(?=.*[a-z])(?=.*[A-Z])(?=(?:.*\\d){2})(?=.*[\\W_]).{8,}$");
        REGEX_MAP.put("Tarjeta1", "^\\d{4}$");
        REGEX_MAP.put("Tarjeta2", "^\\d{4}$");
        REGEX_MAP.put("Tarjeta3", "^\\d{4}$");
        REGEX_MAP.put("Tarjeta4", "^\\d{4}$");
    }
```

### Continue to solve the pivot table

It seems that the problem was that I wrote FOREIGN KEY before specifying the type. It is actually backwards: first I declare the type, then I tell postgres that it is a foreign key. 


In [8]:
plan_subjects_pivot_table = """
DROP TABLE IF EXISTS plan_subjects CASCADE;

CREATE TABLE plan_subjects(
    plan_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (plan_id) REFERENCES study_plans(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
);
"""

conn_to_postgres(plan_subjects_pivot_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


I forgot to put the area of specialization in the subjects table, and as I wanted to update it with the name of the study plan it belongs to (I suppose that is what is meant, maybe it means that it is from the 4 areas of the unam: mathematics, biology, social and humanities). I think there shouldn't be a problem to drop on cascade, as I still do not have data; and when I have it I should store it in an outside csv to not struggle with it later. 

Error: cannot drop table subjects because other objects depend on it
DETAIL:  constraint plan_subjects_subject_id_fkey on table plan_subjects depends on table subjects
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

It seems that I just need to put CASCADE at the end of the command of dropping the table. 

It worked! But while I was dropping the subjects table I got the error 

Error: there is no unique constraint matching given keys for referenced table "study_plans"

And this makes sense to me: how can the database know from all the available names the ones I want to reference? I supposed that when I inserted the data the name itself would look for any direct match with the study_plans table... 

While it makes sense to me that two different subjects cannot share name (if they did, the subject would be the same), it makes sense that postgres is telling me about the lack of specificity. 

I think an answer could be to point it out to my study_plans table. 

### The 4th requirement is

4. Se debe registrar la información académica previa del estudiante como: institución de origen, tipo de bachillerato, promedio del último bachillerato, fecha de egreso, ubicación.

I assume this will be a table one to many (one student to all the attributes in the information). 


In [9]:

student_info_table = """
DROP TABLE IF EXISTS student_info CASCADE;

CREATE TABLE student_info(
    origin VARCHAR(100),
    type_school TEXT,
    average FLOAT,
    graduation DATE,
    location TEXT,
    student_id INTEGER UNIQUE,
    FOREIGN KEY (student_id) REFERENCES students(id)
);
"""
conn_to_postgres(student_info_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


Now I will create the nationalities table. On my Java table I just have two letter codes for countries, so I will just use VARCHAR(2). I did the same mistake as before of not specifying that the code is unique. I need to explicitly put it when I am not referencing a SERIAL id (as SERIAL already ensures that it is impossible for 2 numbers to be equal)

In [10]:
nationalities_table = """ 
DROP TABLE IF EXISTS nationalities CASCADE;

CREATE TABLE nationalities(
    id SERIAL PRIMARY KEY, 
    name VARCHAR(100) NOT NULL,
    code VARCHAR(2) UNIQUE
);
"""

conn_to_postgres(nationalities_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


### Now the fifth point is 

5. Se debe registrar la información académica en la universidad: clave de estudiante, carrera, programa de estudios, fecha de ingreso, semestre actual, estatus (activo, baja temporal, baja definitiva, graduado, titulado), fecha de última actualización, tipo de alumno (regular, irregular), correo institucional, número de créditos obtenidos hasta el momento, método de titulación. El correo institucional es único.

As I created a student info, I think I should also create another table with the university information... I don't know if I should, as student_info is not horribly large. I don't know if there is any consensus on how many attributes should a table have. 

In [11]:
student_college_info_table = """
DROP TABLE IF EXISTS student_college_info CASCADE; 

CREATE TABLE student_college_info(
    student_id PRIMARY KEY, 
    career VARCHAR(100),
    code VARCHAR(10), 
    beginning DATE,
    semester INTEGER,
    status BOOLEAN, 
    regularity BOOLEAN,
    ins_email VARCHAR(20) UNIQUE,
    credits INTEGER, 
    titulation BOOLEAN,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREING KEY (career) REFERENCES study_plans(name)
);
"""



For now I put status and titulation as a boolean type, the problem is that they are not binary decisions. I have heard that postgreSQL allows you to have custom data types. I should probably just pick one that allows me for more options and force the insertion of data on the website to match one of the available options. 

It seems that Enums might be the ones I am looking for, as they restrict the colum to follow a static list of values. 

Composite types are interesting too, as I might use them with the location of a student to simplify the zip code, state, etc. 

I will start with status... the command can go on Python or it necessarily has to 

In [12]:
status_type = """ 
CREATE TYPE status AS ENUM('active', 'temporal_leave', 'definite_leave', 'graduated', 'titulated');
"""

conn_to_postgres(status_type)

Error: type "status" already exists


It is taking quite a while, so it might not work directly from Python. The problem was that I wrote the table subject_plans wrong and that was stopping everything from running. Also I notice that I forgot the quotation marks around the options of ENUM. Now it works! 

I would like to have a visual tool to follow my tables, as they have become quite complex and I am not able to follow their connections in my mind. If I keep struggling I will just draw them out in a piece of paper for reference. I like how they were arranged in Supabase though.

I will continue for now with titulation... the problem is that Mario didn't put the types I should include. There are twelve of them. 

In [13]:
titulation_type = """ 
CREATE TYPE titulation AS ENUM('support_docency',
'investigation_activity',
'amplification_knowledge',
'postgrade_studies', 
'general_exam',
'academic_article',
'titulation_seminar',
'social_service',
'thesis', 
'dissertation',
'academic_level',
'professional_work'
);
"""

conn_to_postgres(titulation_type)

Error: type "titulation" already exists


It is done. Now I have to copy the student_college_info here, as these types need to exist before I create the table. 

In [14]:
student_college_info_table = """
DROP TABLE IF EXISTS student_college_info CASCADE; 

CREATE TABLE student_college_info(
    student_id INTEGER, 
    career_id INTEGER,
    code VARCHAR(10), 
    beginning DATE,
    semester INTEGER,
    status STATUS, 
    regularity BOOLEAN,
    ins_email TEXT UNIQUE,
    credits INTEGER, 
    titulation TITULATION,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (career_id) REFERENCES study_plans(id)
);
"""

conn_to_postgres(student_college_info_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


### The sitxh point is 

6. Se debe guardar el historial de cursos que ha tomado el estudiante: nombre de materia, calificación, semestre en que se cursó la materia, tipo acreditación (ordinaria, extraordinaria). Un alumno no puede inscribirse más de dos veces a un curso.

The thing here is that I don't know what to call this table, as subjects is already taken. Probably taken_subjects is a good name. 

In [15]:
taken_subjects_table = """ 
DROP TABLE IF EXISTS taken_subjects CASCADE;

CREATE TABLE taken_subjects(
    id SERIAL PRIMARY KEY,
    student_id INTEGER, 
    subject_id INTEGER,
    score FLOAT,
    semester INTEGER,
    acreditation BOOLEAN,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
);
"""

conn_to_postgres(taken_subjects_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


I don't understand this error... I have already put the unique tag in subjects

Error: there is no unique constraint matching given keys for referenced table "subjects"

What I don't know is how to prevent that a student is not able to enter two times the same course. First of all this doesn't work in real life (anyone who failed a core subject would be completely uncapable of graduating), but I don't know if it means the same subject at the same semester, as it would make sense to prevent redundancy. Maybe taken_subjects could have a one to many relationship in which it had all the ids of the students that took the class, and force each of them to be unique? But then I would need another pivot table called student_subjects... hadn't I needed pivot tables just for many to many? 

In [16]:
student_subjects_table = """ 
DROP TABLE IF EXISTS student_subjects CASCADE;

CREATE TABLE student_subjects(
    student_id INTEGER,
    subject_id INTEGER,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREING KEY (subject_id) REFERENCES subjects(id)
);
"""



### The seventh point is 

7. Se debe llevar registro de los examenes extraordinarios que han presentado los estudiantes (materia, fecha, calificación).

As this is just one requirement it feels excessive to create a whole new table just for the extraordinary tests... but it also seems clean. I would just need to reference each student and the subject. In case I wanted to see all the test that a student took I would just need to query for its ID

In [17]:
extraordinary_tests_table = """ 
DROP TABLE IF EXISTS extraordinary_tests CASCADE; 

CREATE TABLE extraordinary_tests(
    student_id INTEGER,
    subject_id INTEGER,
    date DATE,
    score FLOAT,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
);
"""

conn_to_postgres(extraordinary_tests_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


### The eigth point says that

8. Se debe llevar registro de los tutores de los estudiantes. Un estudiante sólo puede tener un tutor, pero un profesor puede tener varios tutorados.

As I already have the students table, I should probably a professors table. Just like I did with PAPIME I probably should have a generic role like individual and allow the same table to fill different kind of information depending of it is a professor or a student. 

This works well for authentication, but I don't know if it would make my schema confusing or disorganized. 

For now I will start with the teacher table, as while it is not declared explicitly, at least in the ENES only the teachers can be tutors

Therefore I will jump all the way to point nine

### The ninth requirement is 

9. Se debe llevar registro de las clases que imparte cada profesor. Suponga que cada clase sólo puede ser impartida por un profesor en un semestre dado. Un profesor puede dar varios cursos al semestre.

In [18]:
professors_table = """ 
DROP TABLE IF EXISTS professors CASCADE; 

CREATE TABLE professors(
    id SERIAL PRIMARY KEY,
    first_names VARCHAR(100),
    paternal_surname VARCHAR(100),
    maternal_surname VARCHAR(100)
);
"""

conn_to_postgres(professors_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


I am still confused at when the need for a pivot table becomes necessary, but as one professor can have several subjects, I suppose I need another pivot table... it makes sense though, as one subject can be taught by many professors as well as one professor is able to teach many subjects. 

In [19]:
professors_subjects_table = """ 
DROP TABLE IF EXISTS professors_subjects CASCADE; 
DROP TABLE IF EXISTS classes CASCADE;

CREATE TABLE classes(
    professor_id INTEGER,
    subject_id INTEGER,
    semester INTEGER, 
    FOREIGN KEY (professor_id) REFERENCES professors(id),
    FOREIGN KEY (subject_id) REFERENCES subjects(id)
);
""" 

conn_to_postgres(professors_subjects_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


As professors_subjects is a bit wordy I decided to replace it with classes, adding the semester too. 

I have just noticed that I have ids that are defined simply as INTEGERS while others are defined as SERIAL PRIMARY KEY. I assume the latter is the canonical way to do it and the former is probably preferred when I want to hand-pick the id. 

Now that the professors' tables are done, I can continue with tutors. It seems that people argue about if there is one to many or many to one relationship. 

As I was checking the students table I noticed that I haven't created a type for blood type



In [20]:
blood_type = """ 
CREATE TYPE bloodtype as ENUM(
    'A+',
    'A-',
    'B+',
    'B-',
    'O+',
    'O-',
    'AB+',
    'AB-'
);
"""

conn_to_postgres(blood_type)

Error: type "bloodtype" already exists


And I haven't done it with civil status either...

In [21]:
marital_status_type = """ 
CREATE TYPE maritalstatus AS ENUM(
    'Single', 
    'Married',
    'Divorced',
    'Widowed',
    'Separated'
);
"""

conn_to_postgres(marital_status_type)

Error: type "maritalstatus" already exists


In [22]:
tutors_table = """ 
DROP TABLE IF exists tutors CASCADE; 

CREATE TABLE tutors(
    student_id INTEGER,
    professor_id INTEGER,
    FOREIGN KEY (student_id) REFERENCES students(id),
    FOREIGN KEY (professor_id) REFERENCES professors(id)
);
""" 

conn_to_postgres(tutors_table)

Error: the last operation didn't produce records (command status: DROP TABLE)


### The final requirement is

10. Las tablas de la base deben estar en la tercera forma normal. Si este no es el caso, deberá justificarse el razonamiento para no llevar la tabla a esa forma.

I think I have already done quite a good job respecting 3rd Normal Form, however I just began to have doubts, as I think that the name field that I put on students should be separated by first_name, paternal_name and maternal_name, as that would make querying easier. 

I think I should begin putting the information to the database, but before that I would really like to have a more user-friendly way to visualize my data besides the terminal. I will check if supabase has an integration with GitHub, and if it hasn't, I will just copy and paste the SQL commands directly there. 

# Implementation

## Data

### 2 Study Plans

#### Technology
I need to fill

```SQL
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) UNIQUE,
    code VARCHAR(50),
    duration INTEGER,
    credits INTEGER,
    status BOOLEAN NOT NULL
```

id: 1
name: undergraduate in technology
code: 126
duration: 8
credits: 372
status: 1 (vigente)

#### Neuroscience

id: 2
name: undergraduate in neurosciences
code: 127
duration: 8
credits: 392
status: 1 (vigente)




In [23]:

study_plans_query = """ 
INSERT INTO study_plans(name, code, duration, credits,status)
VALUES
    ('undergraduate in technology', '126', 8, 372, TRUE), 
    ('undergraduate in neurosciences', '127', 8, 392, TRUE)
""" 

conn_to_postgres(study_plans_query)


Error: the last operation didn't produce records (command status: INSERT 0 2)


It seems that when you insert values directly you need to specify if something is a string or not. It seems that the plans do exist now, but why I can't see their values? Do I need to use DBeaver? 

In [24]:
conn_to_postgres("SELECT * FROM study_plans;")

(1, 'undergraduate in technology', '126', 8, 372, True)
(2, 'undergraduate in neurosciences', '127', 8, 392, True)


<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c1a90>

### 10 subjects per study plan

I don't know what I did with subjects, as I took out the connection to postgres and when I returned it marked the error. Next time I will put an X or another uncommon character to find my tables quickly, as having a descriptive name is useful for readability but it is annoying to realize I have 55 'subjects' appearances on this file alone. 

```SQL
    ID SERIAL PRIMARY KEY,
    name VARCHAR(100) UNIQUE,
    theorical_hours INTEGER,
    practical_hours INTEGER,
    hours INTEGER,
    credits INTEGER,
    area VARCHAR(100),
    FOREIGN KEY (area) REFERENCES plan_subjects(plan_id),
    CONSTRAINT hours CHECK (theorical_hours + practical_hours <= hours)
```

#### Technology

##### Calculo I

ID: 1
name: Calculo I 
theorical_hours: 80 
practical_hours: 0 
hours: 80 
credits: 10 
area: Matematicas

##### Algebra lineal

id: 2
name: Algebra Lineal y Geometria Analitica 
theorical_hours: 48 
practical_hours: 0 
hours: 48 
credits: 6 
area: Matematicas

##### Quimica inorganica

id: 3
name: Quimica Inorganica 
theorical_hours: 64 
practical_hours: 32 
hours: 96 
credits: 10 
area: Quimica

##### Biologia general

id: 4
name: Biologia General 
theorical_hours: 64 
practical_hours: 32 
hours: 96 
credits: 10 
area: Biologia

##### Aprendizaje y creatividad

id: 5
name: Tecnicas de Aprendizaje y Creatividad 
theorical_hours: 80 
practical_hours: 0 
hours: 80 
credits: 10 
area: Sociales

##### Calculo II

id: 6
name: Calculo II 
theorical_hours: 80 
practical_hours: 0 
hours: 80 
credits: 10 
area: Matematicas

##### Mecanica clasica

id: 7
name: Mecanica Clasica 
theorical_hours: 64 
practical_hours: 32 
hours: 96 
credits: 10 
area: Fisica

##### Quimica organica

id: 8
name: Quimica Organica 
theorical_hours: 64 
practical_hours: 32 
hours: 96 
credits: 10 
area: Quimica

##### Ecuaciones diferenciales I

id: 9
name: Ecuaciones Diferenciales I 
theorical_hours: 64 
practical_hours: 0 
hours: 64 
credits: 8 
area: Matematicas

##### Computacion I

id: 10
name: Computacion I 
theorical_hours: 48 
practical_hours: 32 
hours: 80 
credits: 8 
area: Computo

#### Neurosciences

##### Matematicas I

id: 11
name: Matematicas I 
theorical_hours: 96 
practical_hours: 0 
hours: 96 
credits: 12 
area: Ciencias Basicas

##### Fisicoquimica

id: 12
name: Fisicoquimica 
theorical_hours: 96 
practical_hours: 0 
hours: 96 
credits: 12 
area: Ciencias Basicas

##### Biologia celular

id: 13
name: Biologia Celular 
theorical_hours: 64 
practical_hours: 0 
hours: 64 
credits: 8 
area: Ciencias Basicas

##### Morfologia de los sistemas

id: 14
name: Morfofisiologia de los Sistemas 
theorical_hours: 96 
practical_hours: 0 
hours: 96 
credits: 12 
area: Neurobiologico

##### Neuroanatomia funcional

id: 15
name: Neuroanatomia Funcional 
theorical_hours: 64 
practical_hours: 0 
hours: 64 
credits: 8 
area: Neurobiologico

##### Histologia y microscopia

id: 16
name: Histologia y Microscopia 
theorical_hours: 0 
practical_hours: 160 
hours: 160 
credits: 10 
area: Ciencias Basicas

##### Biofisica

id: 17
name: Biofisica 
theorical_hours: 64 
practical_hours: 0 
hours: 64 
credits: 8 
area: Ciencias Basicas

##### Matematicas II

id: 18
name: Matemáticas II 
theorical_hours: 64 
practical_hours: 0 
hours: 64 
credits: 8 
area: Ciencias Básicas

##### Bioquimica

id: 19
name: Bioquímica 
theorical_hours: 96 
practical_hours: 0 
hours: 96 
credits: 12 
area: Ciencias Básicas

##### Introduccion a las neurociencias

id: 20
name: Introducción a las Neurociencias 
theorical_hours: 96 
practical_hours: 0 
hours: 96 
credits: 12 
area: Neurobiológico


In [25]:
subjects_query = """ 
INSERT INTO subjects(name, theorical_hours, practical_hours, hours, credits, area, career)
VALUES
    ('Calculo I', 80, 0, 80, 10, 'Matematicas', 1),
    ('Algebra Lineal y Geometria Analitica', 48, 0, 48, 6, 'Matematicas', 1), 
    ('Quimica Inorganica', 64, 32, 96, 10, 'Quimica', 1),
    ('Biologia General', 64, 32, 96, 10, 'Biologia', 1), 
    ('Tecnicas de Aprendizaje y Creatividad', 80, 0, 80, 10, 'Sociales', 1),
    ('Calculo II', 80, 0, 80, 10, 'Matematicas', 1),
    ('Mecanica Clasica', 64, 32, 96, 10, 'Fisica', 1),
    ('Quimica Organica', 64, 32, 96, 10, 'Quimica', 1),
    ('Ecuaciones Diferenciales I', 64, 0, 64, 8, 'Matematicas', 1),
    ('Computacion I', 48, 32, 80, 8, 'Computo', 1),

    ('Matematicas I', 96, 0, 96, 12, 'Ciencias Basicas', 2),
    ('Fisicoquimica', 96, 0, 96, 12, 'Ciencias Basicas', 2),
    ('Biologia Celular', 64, 0, 64, 8, 'Ciencias Basicas', 2),
    ('Morfofisiologia de los Sistemas', 96, 0, 96, 12, 'Neurobiologico', 2),
    ('Neuroanatomia Funcional', 64, 0, 64, 8, 'Neurobiologico', 2),
    ('Histologia y Microscopia', 0, 160, 160, 10, 'Ciencias Basicas', 2),
    ('Biofisica', 64, 0, 64, 8, 'Ciencias Basicas', 2),
    ('Matematicas II', 64, 0, 64, 8, 'Ciencias Basicas', 2),
    ('Bioquimica', 96, 0, 96, 12, 'Ciencias Basicas', 2),
    ('Introduccion a las Neurociencias', 96, 0, 96, 12, 'Neurobiologico', 2);
"""

conn_to_postgres(subjects_query)

Error: the last operation didn't produce records (command status: INSERT 0 20)


It seems that the reason why I can't see the data is that I have not used the fetchall() method on my conn_to_postgres function. 

In [26]:
conn_to_postgres("SELECT name, career FROM subjects WHERE career = 2;")

('Matematicas I', 2)
('Fisicoquimica', 2)
('Biologia Celular', 2)
('Morfofisiologia de los Sistemas', 2)
('Neuroanatomia Funcional', 2)
('Histologia y Microscopia', 2)
('Biofisica', 2)
('Matematicas II', 2)
('Bioquimica', 2)
('Introduccion a las Neurociencias', 2)


<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c25d0>

### 5 regular and irregular students for study plan

I need twenty students on total. As it would be too much effort to find real students I will just create a python loop. 

```SQL
    ID SERIAL PRIMARY KEY,
    first_names VARCHAR(100),
    paternal_surname VARCHAR(100),
    maternal_surname VARCHAR(100), 
    nationality VARCHAR(2),
    curp VARCHAR(30), 
    birth_date DATE,
    email TEXT UNIQUE,
    telephone VARCHAR(15),
    city VARCHAR(100), 
    state VARCHAR(100), 
    street VARCHAR(100),
    external_number INTEGER,
    zip_code INTEGER,
    marital_status MARITALSTATUS,
    blood_type BLOODTYPE,
    NSS INTEGER,
    status STATUS, 
    tutor_id INTEGER,
```

In [52]:
import random
import string 

names = ['Jose', 'Mario', 'Rogelio', 'Yamil', 'Hector', 'Alicia','Paola', 'Valeria', 'Valentina', 'Maria']
surnames = ['Ramirez', 'Martinez', 'Espiritu', 'Gomez', 'Godinez', 'Coronel', 'Herrera', 'Boneta', 'Islas']
nationalities = ['MX', 'DE', 'AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'CA']
cities = ['AG', 'JAL', 'MEX', 'PUE', 'QRO'] # The city will be the capital
# of each of these states
bloodtype = ['A+','A-', 'B+','B-','O+','O-','AB+','AB-']
maritalstatus = ['Single', 'Married', 'Divorced', 'Widowed','Separated']
status_lst = ['active', 'temporal_leave', 'definite_leave', 'graduated', 'titulated']
state_dic = {'AG': 'AS', 
            'JAL': 'JC', 
            'MEX': 'CDMX',
            'PUE': 'PL',
            'QRO': 'QRO'}

def create_data(student = True, plan_id = 1, isregular = True):
    chosen_name = random.choice(names) 
    chs_pat_name = random.choice(surnames)
    chs_mat_name = random.choice(surnames)
    nationality = random.choice(nationalities)
    chs_curp = ''.join(random.choices(string.ascii_uppercase + string.digits, k = 18))
    day = str(random.randint(1, 28))
    if int(day) < 10:
        day = f"0{day}"
    month = str(random.randint(1, 12))
    year = str(2000 + random.randint(0,7))
    birth_date = f"{year}-{month}-{day}"
    email = f"{chosen_name}_{chs_pat_name}_{chs_mat_name}@gmail.com"
    telephone = random.randint(1000000000, 9999999999)
    city = random.choice(cities)
    state = state_dic[city]
    street = random.randint(1,10)
    ext_num = random.randint(1, 999)
    zipcode = random.randint(10000, 99999)
    blood_type = random.choice(bloodtype)
    marital_status = random.choice(maritalstatus)
    nss = random.randint(10000, 99999)
    tutor = random.randint(1, 10)

    if student:
        query_personal_info = """ 
            INSERT INTO students(
                first_names, paternal_surname, maternal_surname, 
                nationality, curp, birth_date, email, telephone,
                city, state, street, external_number, zip_code,
                marital_status, blood_type, NSS, tutor_id)

            VALUES (%s, %s, %s,
            %s, %s, %s, %s, %s, 
            %s, %s, %s, %s, %s, 
            %s, %s, %s, %s)

            RETURNING id; 
        """

        person_data = (chosen_name, chs_pat_name, chs_mat_name, nationality, 
        chs_curp, birth_date, email, telephone, city, state, street, ext_num,
        zipcode, marital_status, blood_type, nss, tutor)

        student_id = conn_to_postgres(query_personal_info, 
        params = person_data, fetch = True)

        query_college_info = """ 
        INSERT INTO student_college_info(
            student_id, career_id, code, beginning, semester, status, 
            regularity, ins_email, credits, titulation)
        
        VALUES(%s, %s, %s, %s, %s, %s,
               %s, %s, %s, %s);
        """

        code = random.randint(100000000, 999999999)
        beginning = str(int(birth_date[:4]) + 20) + birth_date[4:]
        ins_email = f"{chosen_name}.{code}@comunidad.unam.mx"
        status = random.choice(status_lst)

        college_data = (student_id, plan_id, code, beginning, 1,
        status, isregular, ins_email, 0, None)

        conn_to_postgres(query_college_info, college_data)

    else:
        professor_query = """
        INSERT INTO professors(
            first_names,
            paternal_surname,
            maternal_surname)

        VALUES (%s, %s, %s)
        """
        
        professor_data = (chosen_name, chs_pat_name, chs_mat_name)

        conn_to_postgres(professor_query, professor_data)

plan_ids = [1, 2]

for _ in range(10):
    create_data(student = False) # Professors

for plan in plan_ids:
    for _ in range(5):
        create_data(plan_id = plan) # Technology and Neuroscience regulars
    for _ in range(5):
        create_data(plan_id = plan, isregular = False) # Technology and Neuroscience iregulars
 


Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status: INSERT 0 1)
Error: the last operation didn't produce records (command status

I got NameError: name 'string' is not defined from the chs_curp, but I don't understand why. The `string` module indeed exists in python and it is already built in... 

It is not about me picking random.choices instead of random.choice either... I assume I need to import the module, which doesn't sound right, as I have never had trouble using string methods before. 

Importing the module did change the type of error though. Now it says TypeError: str.join() takes no keyword arguments. I know see what it is, I was puting the number of choices that random should take outside of its parentheses. As the join method thought that I was giving him a keyword argument he got confused. 

The other mistake I was having was that I was writing radint instead of ra**n**dint. The n was missing. 

Now I am having trouble with the %s placeholders. It seems that placeholder just works with strings, and that %I is used for an SQL identifier (which I don't think I need, as students and professors have a SERIAL constraint to automatically change the id). 

Now that I think about it I skipped inserting the study_plans info. I am also realizing that I don't have a connection between students and study_plans... Now I see where my confusion lies. I thought that I had mistakenly left out regularity and status from the students table, but the reality is that I did that on purpose, as it is the college_info table the one that checks for regularity, status, and the specific career that each student belongs too. students is just for their personal info (I might change the name later to make it more obvious).

It seems that %L is for a SQL literal, and that is used mainly to prevent mistakes when using single quotes. 

The documentation I am seeing works with FORMAT, which has a quite different structure that the one I tried to use. 

On other source I found that the psycopg library automaitcally converts strings into the necessary data type of the table, and that it always uses %s as a placeholder. 

I will try to work with the study plans first to see if they work. Now that I inserted the data and visualized it (I think I need to get into the habit of doing things more atomically, as I wrote a lot of data without inserting it first) I need to continue with the students data. 

It seems that I can use the RETURNING clause from postgreSQL to return the id of the student and then fill with that same id the college_info table. 

The main problem I encounter is that my conn_to_postgres function automatically closes the connection once a query is executed, and what I want is to both, store any value that RETURNING sent, and also fill the placeholder with the tuple I send. 

I got the error Error: date/time field value out of range: "13/11/2002". With a quick google search I see that the format the data type expects is YYYY-MM-DD. 

I forgot to put the data in the nationalities table

```SQL
    id SERIAL PRIMARY KEY, 
    name VARCHAR(100) NOT NULL,
    code VARCHAR(2) UNIQUE
```

The good thing is that I left the nationalities table fairly short

nationalities = ['MX', 'DE', 'AD', 'AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'CA']

In [28]:
countries_query = """
INSERT INTO nationalities (code, name) 
VALUES
    ('MX', 'Mexico'), ('DE', 'Germany'), 
    ('AD', 'Andorra'), ('AR', 'Argentina'), 
    ('AU', 'Australia'), ('AT', 'Austria'), 
    ('BE', 'Belgium'), ('BO', 'Bolivia'), 
    ('BR', 'Brazil'), ('CA', 'Canada');
"""

conn_to_postgres(countries_query)

Error: the last operation didn't produce records (command status: INSERT 0 10)


In [29]:
conn_to_postgres("SELECT * FROM nationalities")

(1, 'Mexico', 'MX')
(2, 'Germany', 'DE')
(3, 'Andorra', 'AD')
(4, 'Argentina', 'AR')
(5, 'Australia', 'AU')
(6, 'Austria', 'AT')
(7, 'Belgium', 'BE')
(8, 'Bolivia', 'BO')
(9, 'Brazil', 'BR')
(10, 'Canada', 'CA')


<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c26c0>

This error 

```bash
Error: column "regularity" is of type boolean but expression is of type smallint
LINE 7:                $7, $8, $9, $10); 
```

Seems to have been caused because I put 1 on college_data, expecting it to transform the 1 to True, but as the value is sent literally (which makes sense, as if it didn't I would have trouble with the plans, ids, etc) the table is filled incorrectly. 

Now that has got out of the way I got the error `Error: value too long for type character varying(20)` but on students I don't have any attribute that has VARCHAR(20). 

If I want to "solve" the problem quickly I can just use TEXT, as it seems is literally the same as VARCHAR but the latter forces itself to n characters. An unexpectedly long string could be of the email, so I think that will be the reason my code is breaking. 

Indeed `ins_email VARCHAR(20) UNIQUE` has only 20 characters. 

Now I just have `Error: the last operation didn't produce records (command status: INSERT 0 1)` which I don't think it is a problem, as it is just telling me that there wasn't anything "printed" (this is a direct consequence of the update to the RETURNING clause in my conn_to_postgres function). 

Lets see if everything is all right. 


In [30]:
conn_to_postgres("SELECT * FROM students;")

(1, 'Jose', 'Herrera', 'Godinez', 'AD', 'RFI9WPAKGGWQALQP5P', datetime.date(2007, 12, 20), 'Jose_Herrera_Godinez@gmail.com', '8198410122', 'JAL', 'JC', '2', 613, 21851, 'Separated', 'O+', 95511, 5)
(2, 'Rogelio', 'Herrera', 'Martinez', 'AT', 'G1SX62KY4CLFXBNWPL', datetime.date(2003, 9, 2), 'Rogelio_Herrera_Martinez@gmail.com', '1030509706', 'QRO', 'QRO', '6', 668, 73376, 'Separated', 'B-', 15539, 1)
(3, 'Mario', 'Herrera', 'Coronel', 'BO', 'KO8IFOZQ8XCZ9SKHQW', datetime.date(2002, 8, 19), 'Mario_Herrera_Coronel@gmail.com', '6403013722', 'AG', 'AS', '1', 396, 86218, 'Single', 'O-', 41360, 1)
(4, 'Hector', 'Boneta', 'Herrera', 'BE', 'HVQGZHNOWZ70N5LIZ2', datetime.date(2003, 8, 8), 'Hector_Boneta_Herrera@gmail.com', '6109698494', 'PUE', 'PL', '8', 576, 32812, 'Separated', 'O-', 26811, 5)
(5, 'Paola', 'Godinez', 'Gomez', 'AU', 'T1A9BRDGVZEJ5E0Z1P', datetime.date(2006, 3, 26), 'Paola_Godinez_Gomez@gmail.com', '5609694245', 'PUE', 'PL', '7', 697, 36486, 'Single', 'O+', 35868, 1)
(6, 'Paola',

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c2a80>

Now the data is correct! In fact, as I ran the code many times I have up to 102 students. Lets check professors now. 

In [31]:
conn_to_postgres("SELECT * FROM professors;")

(1, 'Yamil', 'Boneta', 'Coronel')
(2, 'Jose', 'Islas', 'Boneta')
(3, 'Hector', 'Ramirez', 'Herrera')
(4, 'Valentina', 'Ramirez', 'Godinez')
(5, 'Valentina', 'Godinez', 'Islas')
(6, 'Rogelio', 'Boneta', 'Godinez')
(7, 'Hector', 'Boneta', 'Ramirez')
(8, 'Valeria', 'Godinez', 'Martinez')
(9, 'Jose', 'Herrera', 'Boneta')
(10, 'Valentina', 'Godinez', 'Herrera')


<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c32f0>

And indeed I have now up to 40 professors. 

Lets check if I have 5 regular and 5 irregular students in both plans. 

In [32]:
conn_to_postgres("SELECT * FROM students WHERE career_id = 1 isregular = TRUE;")
conn_to_postgres("SELECT * FROM students WHERE career_id = 1 isregular = FALSE;")
conn_to_postgres("SELECT * FROM students WHERE career_id = 2 isregular = TRUE;")
conn_to_postgres("SELECT * FROM students WHERE career_id = 2 isregular = FALSE;")

Error: syntax error at or near "isregular"
LINE 1: SELECT * FROM students WHERE career_id = 1 isregular = TRUE;
                                                   ^
Error: syntax error at or near "isregular"
LINE 1: SELECT * FROM students WHERE career_id = 1 isregular = FALSE...
                                                   ^
Error: syntax error at or near "isregular"
LINE 1: SELECT * FROM students WHERE career_id = 2 isregular = TRUE;
                                                   ^
Error: syntax error at or near "isregular"
LINE 1: SELECT * FROM students WHERE career_id = 2 isregular = FALSE...
                                                   ^


### 2. La base de datos debe tener 3 tipos de usuarios:

#### a. administrador con permisos totales

I already have the schronding user that can has total permissions, but I think I will create another user called admnistrator with a specific password in order to be explicit about the requirement. 

There seems to be multiple attributes, and even an interactive mode to create the roles more easily. 

It was effortless, I just needed to say that the administrator role was a superuser... the thing is that I wonder if it is able to modify all my tables or just UNAM_database... I will probably modify all, as I didn't specified what tables should act on

```bash
sudo -i -u postgres
[sudo] password for schronding: 
postgres@debian:~$ createuser --interactive --pwprompt administrator
Enter password for new role: 
Enter it again: 
Shall the new role be a superuser? (y/n) y
```

#### b. servicios escolares con permisos de lectura para todas las tablas y escritura sólo actualizar e ingresar nuevos usuarios, pero no para eliminar.

This seems more challenging. I think these should not have the CREATEDB role attribute, but how could I prevent it from eliminating the data?...

It seems that postgres doesn't have delete by default, I can just left it out and the problem is solved. 

I will name the role school_services. 

```bash
sudo -u postgres psql
[sudo] password for schronding: 
could not change directory to "/home/schronding/repos/UNAM_database": Permission denied
psql (15.15 (Debian 15.15-0+deb12u1))
Type "help" for help.

postgres=# CREATE ROLE school_services LOGIN PASSWORD '987654321';
CREATE ROLE
postgres=# \du
postgres=# \l
postgres=# GRANT SELECT, INSERT, UPDATE ON unam_database TO school_services; 
ERROR:  relation "unam_database" does not exist
```

I have doubt here. The documentation I found talked about using a table_name, not a whole database. It makes sense that school_services can only act on the tables related to students... but as I have created quite a lot of tables, I would need to grant access to each of them. 

It seems that indeed I need to use the CONNECT clause along the database name. I also need to grant usage on the public schema. 

```bash
postgres=# GRANT CONNECT ON DATABASE unam_database TO administrator;
GRANT
postgres=# GRANT USAGE ON SCHEMA public TO administrator; 
GRANT
postgres=# GRANT CONNECT ON DATABASE unam_database TO school_services;
GRANT
postgres=# GRANT USAGE ON SCHEMA public TO school_services; 
GRANT
```

Now lets try again and see if it works... I need to put the name of the tables that it has access. 

I have a repeated table 

```bash

                 List of relations
 Schema |         Name         | Type  |   Owner    
--------+----------------------+-------+------------
 public | classes              | table | schronding
 public | extraordinary_tests  | table | schronding
 public | nationalities        | table | schronding
 public | plan_subjects        | table | schronding
 public | professors           | table | schronding
 public | student_college_info | table | schronding
 public | student_info         | table | schronding
 public | students             | table | schronding
 public | study_plan           | table | schronding
 public | study_plans          | table | schronding
 public | subjects             | table | schronding
 public | taken_subjects       | table | schronding
 public | tutors               | table | schronding
 ```

It seems that it is just a table I forgot to delete. 

It seems it is done

```bash
sudo -u postgres psql
\c unam_database
unam_database=# GRANT INSERT, UPDATE, SELECT, DELETE ON classes, extraordinary_tests, nationalities, plan_subjects, professors, student_college_info, student_info, students, study_plans, subjects, taken_subjects, tutors TO administrator;
GRANT
```

In [33]:
#conn_to_postgres("DROP TABLE IF EXISTS study_plan CASCADE;")

Now that study_plan is eliminated I know I have twelve tables. 

Lets do the same with school_services... should I really grant the INSERT, UPDATE and SELECT permissions on all the tables? The ones that seem relevant to them are:

extraordinary_tests, student_college_info, student_info, students, and taken_subjects

the other tables seem more concerned with professors and coordinators. 

#### c. coordinador de carrera con permisos de lectura sólo sobre la información de los alumnos de la carrera correspondiente y la información de todos los profesores.

In order for this to work I would need to limit the access of each of the coordinators (as I have to study plans) to just the id of their career. I think I could use a view to acocmplish this, as I could just use the WHERE clause with career_id = 1 (or two, if it was the coordinator of neurosciences). 

In [34]:
technology_view = """ 
CREATE VIEW technology_students AS
SELECT *
FROM student_college_info
WHERE career_id = 1;
"""

neurosciences_view = """ 
CREATE VIEW neuroscience_students AS
SELECT *
FROM student_college_info
WHERE career_id = 2;
"""

conn_to_postgres(technology_view)
conn_to_postgres(neurosciences_view)

Error: the last operation didn't produce records (command status: CREATE VIEW)
Error: the last operation didn't produce records (command status: CREATE VIEW)


Now I would just need to grant select on the view... but here is where I wonder if my decision to separate the college info into a separate table was wise. Can the coordinator access students with just the ids?  

For now I will create the roles for the coordinators. 

```bash
unam_database=# CREATE ROLE technology_coordinator WITH LOGIN PASSWORD '';
CREATE ROLE
unam_database=# CREATE ROLE neurosciences_coordinator WITH LOGIN PASSWORD '';
CREATE ROLE
```

I didn't put the passwords for terms of security. 

It seems that the correct way to display all the information that the coordinator needs I should use the JOIN clause. 


In [35]:
technology_view = """ 
CREATE OR REPLACE VIEW technology_students AS
SELECT
    t1.id,
    t1.first_names,
    t1.paternal_surname,
    t1.maternal_surname, 
    t2.semester,
    t2.status,
    t2.regularity,
    t2.credits,
    t2.titulation
FROM students t1
JOIN student_college_info t2 ON t1.id = t2.student_id
WHERE t2.career_id = 1; 
"""

neurosciences_view = """ 
CREATE OR REPLACE VIEW neuroscience_students AS
SELECT
    t1.id,
    t1.first_names,
    t1.paternal_surname,
    t1.maternal_surname, 
    t2.semester,
    t2.status,
    t2.regularity,
    t2.credits,
    t2.titulation
FROM students t1
JOIN student_college_info t2 ON t1.id = t2.student_id
WHERE t2.career_id = 2; 
"""

conn_to_postgres(technology_view)
conn_to_postgres(neurosciences_view)

Error: cannot drop columns from view
Error: cannot drop columns from view


Now that the coordinators have their respective views for their students I will give both coordinators the power to see all the professors.

```SQL
GRANT SELECT ON professors TO technology_coordinator, neurosciences_coordinator;
GRANT
```

It seems I am done with point 2 too! As the page seems like a complete work on its own, I think I can work with putting the step 3.5, that is related more to the database that on the web page. 

I think I could use the trigger to raise an exception where a student cannot inscribe more than two times to a single subject (less or equal than 2). 

This seems to also make use of a function (I suppose Mario is talking about a SQL function, as in python I already have one). 

In [36]:
check_attempts_limit_query = """
CREATE OR REPLACE FUNCTION check_attempts_limit()
RETURNS TRIGGER AS $$
DECLARE
    attempt_count INTEGER;
BEGIN
    SELECT COUNT(*) INTO attempt_count
    FROM taken_subjects
    WHERE student_id = NEW.student_id
    AND subject_id = NEW.subject_id;

    IF taken_subjects.score >= 6 THEN
        RAISE EXCEPTION 'The student has already passed this subject
            with the score: %', NEW.score;
    END IF; 
    
    IF attempt_count >= 2 THEN
        RAISE EXCEPTION 'The student with id:% has already taken the subject:% twice.', NEW.student_id, NEW.subject_id;
    END IF;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql; 
"""

conn_to_postgres(check_attempts_limit_query)

Error: the last operation didn't produce records (command status: CREATE FUNCTION)


My problem here is that I don't know what the correct logic is to store a tuple that combines a specific subject_id to a student_id. I think the problem lies in that on the taken_subjects table I just have students, how could I compare it with subjects if I don't have it there? 

It seems that creating another variable was also incorrect, as the way you reference the variable of triggers is with NEW. What NEW contains is basically the info that is used to trigger the function, in this case, the student and subject ids. While the $$ confused me, they seem to act just like a great quote (like the """ in python) so I don't need to escape every character for the function to work. 

I got the error 

```Bash
Error: unrecognized exception condition "The student with id:% has already taken the 
        subject:% "
CONTEXT:  compilation of PL/pgSQL function "check_attempts_limit" near line 11
```

Both when I used % and %s. The jump line didn't do anything nor the missing _id on the final of the AND line. When I changed the double quotes with single quotes the problem dissapeared. I suppose that double quotes messed up with the long string that you get when you put triple double quotes. 

Now I need to link this function to the taken_subjects table. 

In [37]:
trigger_two_courses_exception_query = """ 
CREATE TRIGGER trigger_check_attempts
BEFORE INSERT ON taken_subjects
FOR EACH ROW
EXECUTE FUNCTION check_attempts_limit();
"""

conn_to_postgres(trigger_two_courses_exception_query)

Error: the last operation didn't produce records (command status: CREATE TRIGGER)


In case this function isn't enough (as I don't even know if it is possible to create a trigger without a function) I will create a function that sums all the credits of a student (and I might put that on the credits attribute of student_college_info)

In [38]:
sum_credit_function_query = """ 
CREATE OR REPLACE FUNCTION calculate_total_credits(target_student_id INTEGER)
RETURNS INTEGER AS $$
DECLARE
    total INTEGER;
BEGIN
    SELECT COALESCE(SUM(s.credits), 0) INTO total
    FROM taken_subjects ts
    JOIN subjects s ON ts.subject_id = s.id
    WHERE ts.student_id = target_student_id
    AND ts.score >= 6;

    RETURN total;
END;
$$ LANGUAGE plpgsql; 
"""

conn_to_postgres(sum_credit_function_query)

Error: the last operation didn't produce records (command status: CREATE FUNCTION)


I got several errors on this function. The main problem was that the first return on the second line is plural, so I need to write RETURNS. The noun is used for the actual data that the function creates. There are way more semicolons in function creation, but the pattern I can distinguish now is that you use ; for every big command, even if they seem "redundant" such as END; 

I need to pass this function to other that updates the credits of the students. 

In [39]:
update_student_credits_function_query = """ 
CREATE OR REPLACE FUNCTION update_student_credits()
RETURNS TRIGGER AS $$
DECLARE
    new_total INTEGER;
BEGIN
    new_total := calculate_total_credits(NEW.student_id);

    UPDATE student_college_info
    SET credits = new_total
    WHERE student_id = NEW.student_id;

    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
""" 

conn_to_postgres(update_student_credits_function_query)

Error: the last operation didn't produce records (command status: CREATE FUNCTION)


Another interesting rule of thumb is that it works related to JavaScript. When you declare a new variable, you need to use ; as that is a statement too. 

Now we just simply create the trigger. It didn't made sense that I used before, as I would want to know my credits once I accredit a subject, wouldn't I? 

In [40]:
trigger_total_student_credit_query = """ 
CREATE TRIGGER trigger_update_credits
AFTER INSERT OR UPDATE OR DELETE ON taken_subjects
FOR EACH ROW
EXECUTE FUNCTION update_student_credits();
"""

conn_to_postgres(trigger_total_student_credit_query)

Error: the last operation didn't produce records (command status: CREATE TRIGGER)


Now lets try putting some data and see if the credits update. I suppose I should insert the subject into taken subjects. 

In [41]:
conn_to_postgres("SELECT * FROM taken_subjects")

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c3200>

Why does `<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7f75ea78fa70>` appear if I have already updated the conn_to_postgres function? I didn't tell the function to fetch data, so the .fetchall() method should have gone through. 

The random thought of the possibility of a technology student putting a subject of neurosciences or viseversa is worrying me. I know I can protect against that data mistake on the webpage, but have I done enough to protect it on my SQL tables? 

In [42]:
match_student_career_test = """
INSERT INTO taken_subjects (student_id, subject_id, score, semester, acreditation) 
VALUES (1, 20, 10, 1, TRUE);
"""
conn_to_postgres(match_student_career_test)

Error: missing FROM-clause entry for table "taken_subjects"
LINE 1: taken_subjects.score >= 6
        ^
QUERY:  taken_subjects.score >= 6
CONTEXT:  PL/pgSQL function check_attempts_limit() line 10 at IF


How can I not have id 1 if I have already inserted like 100 students? 

```bash
Error: insert or update on table "taken_subjects" violates foreign key constraint "taken_subjects_student_id_fkey"
DETAIL:  Key (student_id)=(1) is not present in table "students".
```

It seems that the ids of students kept ramping up while also deleting the ones that weren't inserted correctly. I can query for the available ids directly though

In [43]:
conn_to_postgres("SELECT id, first_names, paternal_surname FROM students ORDER BY id ASC LIMIT 10")

(1, 'Jose', 'Herrera')
(2, 'Rogelio', 'Herrera')
(3, 'Mario', 'Herrera')
(4, 'Hector', 'Boneta')
(5, 'Paola', 'Godinez')
(6, 'Paola', 'Espiritu')
(7, 'Hector', 'Ramirez')
(8, 'Maria', 'Herrera')
(9, 'Yamil', 'Herrera')
(10, 'Mario', 'Coronel')


<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c3b60>

That wasn't actually the problem, as I did got id 1 to 10 (I don't think this is simply the relative order, as in the query I put specificially id).

Now lets check if I have a technology student with a neuroscience subject. 

In [44]:
conn_to_postgres("SELECT * FROM taken_subjects")

<psycopg.Connection [IDLE] (host=localhost user=schronding database=unam_database) at 0x7fd6e21c37a0>

(1, 1, 20, 10.0, 1, True)
(2, 1, 20, 10.0, 1, True)

It seems that I do, and not only that, but that a student passed two times the same subject. Even though Mario did not ask for this, I know this behavior is incorrect. I tried a third time and it seems that the trigger worked! 

```bash
Error: The student with id:1 has already taken the subject:20 twice.
CONTEXT:  PL/pgSQL function check_attempts_limit() line 11 at RAISE
```

Lets modify the function to also include the possibility that if you get more than 6, you cannot inscribe that subject again either. Also I will try to include the whole name and the name of the subject too, as with the ids is simply not clear. 

I will try to prevent the students from taking subjects outside their careers with the following function. 

In [45]:
match_student_career_function_query = """ 
CREATE OR REPLACE FUNCTION check_match_student_career()
RETURNS TRIGGER AS $$
DECLARE 
    student_career INTEGER;
    subject_career INTEGER;
BEGIN
    SELECT career_id INTO student_career
    FROM student_college_info
    WHERE student_id = NEW.student_id; 

    SELECT career INTO subject_career
    FROM subjects
    WHERE subject_career = NEW.subject_id; 

    IF student_career != subject_career THEN
        RAISE EXCEPTION 'There is a mistmatch. The student is in career
        % but the subject is from career %', NEW.student_id, NEW.subject_id;
    END IF; 
    
    RETURN NEW;
END;
$$ LANGUAGE plpgsql; 
"""

conn_to_postgres(match_student_career_function_query)

Error: the last operation didn't produce records (command status: CREATE FUNCTION)


It seems it worked as it is talking about an error to produce records, not of syntax `Error: the last operation didn't produce records (command status: CREATE FUNCTION)`

While it might have compiled, the logic was flawed, 

Lets check now if the trigger for total credits work. 

In [46]:
conn_to_postgres("SELECT credits FROM student_college_info WHERE student_id =1", fetch=True)

0

It worked! But indeed, it is wrong, as that person got double credits by passing the same subject twice. 

It seems that procedures are similar to functions, but are mainly used for transactions with COMMIT and ROLLBACK. 

One procedure that comes to mind is to graduate a student when it has reached enough credits and he has a titulation method too. 

In [47]:
graduation_procedure_query = """ 
CREATE OR REPLACE PROCEDURE graduate_student(target_student_id INTEGER) 
AS $$
DECLARE 
    current_credits INTEGER;
    student_career INTEGER; 
    titulation_status TITULATION; 
BEGIN 
    SELECT credits, career, titulation 
    INTO current_credits, student_career, titulation_status
    FROM student_college_info
    WHERE target_student_id = student_id;

    IF current_credits >= 392
    AND titulation_status != NULL 
    AND student_college_info.career_id = 2 THEN
        UPDATE student_college_info
        SET status = 'graduated' 
        WHERE target_student_id = student_id; 
    END IF; 

    IF current_credits >= 372
    AND titulation_status != NULL 
    AND student_college_info.career_id = 1 THEN
        UPDATE student_college_info
        SET status = 'graduated' 
        WHERE target_student_id = student_id; 
    END IF; 

    COMMIT; 
    
END;
$$ LANGUAGE plpgsql     
"""

conn_to_postgres(graduation_procedure_query)

Error: the last operation didn't produce records (command status: CREATE PROCEDURE)


It seems I misspeled the language `Error: language "pgsql" does not exist`, it is actually `plpgsql`

Now I got the error 
```bash
Error: syntax error at or near "FROM"
LINE 7:     FROM taken_subjects
            ^
```

Now that I have a fresh mind I also see that I put the WHERE FROM logic inside the DECLARE block, which is incorrect, as I should put it on the BEGIN (the body of the function) block. 

I don't understand why I am getting the error 
```bash
Error: syntax error at or near "END"
LINE 18:     END IF; 
             ^
```

Checking at the documentation it seems that I need to be explicit everywhere. While I declared above that the table I am interested is student_college_info I am not declaring in which id is 'graduated' supposed to be set. 

Now that I think about it I will need to put a new invented subject with 300 credits, as with the ones I have now is not enough for anyone to graduate. 

I am also noticing that the syntax is not SET TO nor var = value, but that I need the clause UPDATE and that I need to write which specific table I am going to update. Once that is done, I don't need to the use the dot notation to reference which attribute I want to modify, I can just write it directly. 

It seems to work now, and I have comprobated that the order in which I put student_id = target_student_id doesn't matter. While it is confusing, in this case the equal sign (=) is performing a comparisson, so it is inconsequential if I wrote as previously or as target_student_id = student_id. 

Now I can finally work on to the web page. As I am very short of time, I have encountered a python library called `streamlit` that seems to be able to facilitate the publishing of pages by a large margin. 

In [48]:
import streamlit as st 
import pandas as pd 
import psycopg 
import os 
from streamlit_jupyter import StreamlitPatcher
from dotenv import load_dotenv 

load_dotenv()
StreamlitPatcher().jupyter()

Now that the necessary libraries are set, I will need to have a simple layout. It seems that there is a built-in connection to databases

In [49]:
st_conn = st.connection("postgresql", type = "sql")
all_students = st_conn.query("SELECT * FROM students;")

2026-01-21 18:00:19.686 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 18:00:20.416 No runtime found, using MemoryCacheStorageManager
2026-01-21 18:00:20.420 No runtime found, using MemoryCacheStorageManager
2026-01-21 18:00:20.422 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


It seeems I need to download `sqlalchemy` and `pypi_package` in order for the connection to work. 

I was wondering on when I would need to choose the data to acess the specific database, but errors are extremely clear with streamlit 

```bash
StreamlitAPIException: Missing SQL DB connection configuration. Did you forget to set this in `secrets.toml` or as kwargs to `st.connection`?
```

It seems I need to move my credentials to secrets.toml, which for what I understand is similar to JSON. 

Now that I have wrote the information for the connections it seems it is still not enough, as the error remains. I wonder if this has to do with the fact I am working with a Jupyter environment... or maybe the convention `app.py` is a given. 

It seems that the compatibility is not given, as someone created a library for them to work together. I need the command `pip install streamlit_juypter`. 

It still says that it is missing the DB connection... 

it seems that my problem was that I needed to create a hidden folder called .streamlit and secrets.toml should be inside it. 

It also seems to be forced with lowercase, as I indeed wrote dialect but it didn't detected it. The conventions are database and username too. 

It seems that this works with psycop2g, not with psycopg. 

I have made progress! I am getting a different error now

```bash
2026-01-21 12:17:32.003 No runtime found, using MemoryCacheStorageManager
2026-01-21 12:17:32.005 No runtime found, using MemoryCacheStorageManager
2026-01-21 12:17:32.006 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
```



streamlit offers quite a big variety of pre-built web page blocks. 

In [50]:
st.title("UNAM Database")
st.sidebar.selectbox("Test", ["Option A", "Option B"])

# UNAM Database

2026-01-21 18:00:20.593 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 18:00:20.623 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 18:00:20.633 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 18:00:20.645 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 18:00:20.646 Session state does not function when running a script without `streamlit run`
2026-01-21 18:00:20.648 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


2026-01-21 18:00:21.540 
  command:

    streamlit run /home/schronding/repos/UNAM_database/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
2026-01-21 18:00:21.545 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-21 18:00:21.559 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


'Option A'

It seems I need to be explicit in order to have the page render on the browser

```bash
Warning: to view this Streamlit app on a browser, run it with the following
  command:

    streamlit run /home/schronding/repos/UNAM_database/.venv/lib/python3.13/site-packages/ipykernel_launcher.py [ARGUMENTS]
```

In [51]:
#st.run("/home/schronding/repos/UNAM_database/.venv/lib/python3.13/site-packages/ipykernel_launcher.py")

The reason why the previous command didn't worked is that the command wasn't missing parenthesis and dots like the traditional python syntax, but that this was meant to be run on the terminal. While I see the browser tab now, I got errors 

```bash
To connect another client to this kernel, use:
    --existing kernel-19104.json
[IPKernelApp] ERROR | Unable to initialize signal:
Traceback (most recent call last):
  File "/home/schronding/repos/UNAM_database/.venv/lib/python3.13/site-packages/ipykernel/kernelapp.py", line 720, in initialize
    self.init_signal()
    ~~~~~~~~~~~~~~~~^^
  File "/home/schronding/repos/UNAM_database/.venv/lib/python3.13/site-packages/ipykernel/kernelapp.py", line 556, in init_signal
    signal.signal(signal.SIGINT, signal.SIG_IGN)
    ~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/schronding/miniconda3/lib/python3.13/signal.py", line 58, in signal
    handler = _signal.signal(_enum_to_int(signalnum), _enum_to_int(handler))
ValueError: signal only works in main thread of the main interpreter
```

As it seems that this was the first attempt of the developer to enter the open source world, it might be buggy. I might just build the `app.py` archive and put my comments on there with #